In [ ]:
import requests
import json
from tqdm import tqdm
import time
import os
import random

URLs

- http://13.215.146.238/api/api.php/select_province_and_zone

This URL will return a list of all the provinces and election zones in the country.

- http://13.215.146.238/api/api.php/select_election_zone_candidates?province=71&zoneNumber=2

This URL will return the details of all the election candidates in the province with the ID `71` and zone number `2`.

- http://13.215.146.238/api/api.php/select_election_candidate?candidateId=e8ce23553399625dcf33b029d29c2084

This URL will return the details of the election candidate with the ID `e8ce23553399625dcf33b029d29c2084`.


- http://13.215.146.238/images/candidate/thumb/71/2/7f2ebf81374ec54fd727f65d882c88d3.jpg

This URL will return a thumbnail image of the election candidate with the ID `e8ce23553399625dcf33b029d29c2084`.

In [36]:
def getter(slug, params={}):
    ep = 'http://13.215.146.238/api/api.php/' + slug
    resp = requests.get(ep, params=params)
    data = resp.json()
    if 'MessageCode' in data.keys() and data['MessageCode'] == '00000':
        return data['Result']
    if 'error_code' in data.keys() and data['error_code'] == 0:
        return data['data_list']
    return []

In [20]:
def get_candidate_info(candidate_id):
    return getter('select_election_candidate',
                  {'candidateId': candidate_id})

In [21]:
get_candidate_info('e8ce23553399625dcf33b029d29c2084')

{'Id': 'e8ce23553399625dcf33b029d29c2084',
 'Title': 'นางสาว',
 'FirstName': 'อรอนงค์',
 'LastName': '  กาญจนชูศักดิ์',
 'PartyName': 'ประชาธิปัตย์',
 'PartyNumber': '1',
 'CandidateNo': '1',
 'Age': '61',
 'Occupation': 'นักการเมือง',
 'HighestEducation': 'ปริญญาโท'}

In [34]:
def get_candidates_in_district(prov_code, elect_dist_number):
    return getter('select_election_zone_candidates',
                  {'province': prov_code, 'zoneNumber': elect_dist_number})

In [39]:
get_candidate_info('7f2ebf81374ec54fd727f65d882c88d3')

{'Id': '7f2ebf81374ec54fd727f65d882c88d3',
 'Title': 'นาย',
 'FirstName': 'ยรรยง',
 'LastName': '  กาญจนชูโต',
 'PartyName': 'รวมไทยสร้างชาติ',
 'PartyNumber': '229',
 'CandidateNo': '1',
 'Age': '63',
 'Occupation': 'อดีตกำนัน',
 'HighestEducation': 'ปริญญาตรี'}

In [ ]:
resp = requests.get('http://13.215.146.238/images/candidate/thumb/71/2/7f2ebf81374ec54fd727f65d882c88d3.jpg')

In [40]:
province_list = getter('select_province_and_zone')

In [53]:
def candidate_list_from_api():
    candidate_list = []
    for province in tqdm(province_list):
        dist_nums = len(province['zone_list'])
        prov_code = province['province_code']
        for dist_num in range(1, dist_nums+1):
            candidates = get_candidates_in_district(prov_code, dist_num)
            for candidate in candidates:
                candidate['ProvinceName'] = province['province_name']
                candidate['ProvinceCode'] = prov_code
                candidate['ZoneNumber'] = dist_num
            candidate_list += candidates
    return candidate_list

In [54]:
candidate_list = candidate_list_from_api()

100%|███████████████████████████████████████████| 77/77 [00:45<00:00,  1.70it/s]


In [59]:
def save_candidate_file(candidate_list):
    with open('smartvote-candidates.json','w') as fp:
        json.dump(candidate_list, fp, ensure_ascii=False)

In [60]:
save_candidate_file(candidate_list)

In [64]:
for i, candidate in tqdm(enumerate(candidate_list)):
    if 'Age' in candidate.keys(): continue
    candidate_id = candidate['CandidateId']
    new_info = get_candidate_info(candidate_id)
    candidate.update(new_info)
    
    # sleep
    time.sleep(random.randrange(10,300)/1000)
    
    if random.random() < 0.01:
        time.sleep(1)

    if i % 100 == 0 or i == len(candidate_list) - 1:
        save_candidate_file(candidate_list)

4767it [03:07, 25.41it/s]   


In [65]:
with open('smartvote-candidates.json') as fp:
    candidate_list = json.load(fp)

# Save

In [66]:
import pandas as pd

In [67]:
candidate_df = pd.DataFrame(candidate_list).drop(columns='Id')

In [69]:
candidate_df

,CandidateId,Title,FirstName,LastName,PartyName,PartyNumber,CandidateNo,Votable,ProvinceName,ProvinceCode,ZoneNumber,Age,Occupation,HighestEducation
0,4eaa05d22e5210ff55be06bea28f00b5,นาย,เตชิต,ดำดี,ไทยสร้างไทย,227,1,True,กระบี่,81,1,59,ธุรกิจส่วนตัว,ปริญญาตรี
1,8e5cce4ea6265107a8f86e54812adf5d,นาย,กิตติ,กิตติธรกุล,ภูมิใจไทย,63,2,True,กระบี่,81,1,42,เลขานุการนายกองค์การบริหารส่วนจังหวัดกระบี่,ปริญญาโท
2,ed0aa9d9423f60a85589beba5fa30e8c,นาย,ธนวัช,ภูเก้าล้วน,ประชาธิปัตย์,1,3,True,กระบี่,81,1,43,นักธุรกิจ,ปริญญาโท
3,8024dbdb7ccaabeffc13c1a28465bac8,นาย,สรรเพ็ชญ,ศรีสวัสดิ์,รวมไทยสร้างชาติ,229,4,True,กระบี่,81,1,29,ธุรกิจส่วนตัว,ปริญญาตรี
4,af6604ed85484d4600d7ac58027d0ee6,ร้อยตำรวจตรี,ฮัจญีวิทย์,แดงกุล,เสรีรวมไทย,145,5,True,กระบี่,81,1,55,รับราชการ,ปริญญาตรี
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,64cf930dc5715e70c157b69e049d8e30,นาย,สุวีศักดิ์,แสงสาย,ไทยภักดี,234,11,True,อุบลราชธานี,34,11,50,รับจ้าง,ปริญญาโท
4763,c49190208de4ab14a4db5731e7a08aee,นาย,ฉัตรชัย,คำใส,ไทยก้าวหน้า,246,12,True,อุบลราชธานี,34,11,63,ทนายความ,ปริญญาตรี
4764,2e95ebe8f28ddeac52621c05b6457e03,นาย,นพณัช,นามสีหาญ,ช่วยชาติ,157,13,True,อุบลราชธานี,34,11,54,เกษตรกร,ต่ำกว่าปริญญาตรี
4765,c16d025f19810b8ebf9676857326ccf0,นาย,บุญมา,จันทะรักษา,พลังปวงชนไทย,184,14,True,อุบลราชธานี,34,11,73,ทำนา,ต่ำกว่าปริญญาตรี


In [68]:
candidate_df.to_csv('smartvote-candidates.csv')